In [1]:
from bokeh.models import ColumnDataSource, Legend
from bokeh.plotting import figure, show
from bokeh.models.ranges import FactorRange
from bokeh.core.properties import value
import holoviews as hv
import intake
import pandas as pd
# hv.extension('bokeh')


In [2]:
# %%writefile ../visualization/plotting.py

def plotAtLocation(df, location):
    baseCols = ["Type", "Month"]
    locationDict = {"Magnolia & Alice": "East Knoxville Recycling Center", 
                    "225 Moody": "South Knoxville Recycling Center",
                    "4440 Western Av.": "North Knoxville Recycling Center",
                    "341 Parkvillage": "West Knoxville Recycling Center",
                    "227 Willow Av.": "Downtown Knoxville Recycling Center",
                    "Curbside City-Wide": "Curbside City-Wide Pickup",
                    "Downtown": "Downtown Pickup",
                    "KPD": "KPD", # what is this?
                    "Recycling Trailer": "Recycling Trailer"}
    cols = baseCols+[location]
    locationDF = df[cols]
    locationDF.fillna(0)
    locationDF = locationDF.pivot(index='Month', columns='Type')
    source = ColumnDataSource(locationDF)
    colNames = source.column_names
    if location in locationDict:
        locationName = locationDict[location]
    else:
        locationName = location
        print(location, "not in dict")

    plot = figure(plot_width=800, plot_height=400, 
                  title="Weight of Waste: {}".format(locationName), 
                  x_axis_type="datetime")
    legendItems = []
    for i, colName in enumerate(colNames[1:]):
        groupname, varname = colName.split("_")
        currentLine = plot.line(x='Month', y=colName, source=source, color=colors[i], line_width=2)
        plot.xaxis.axis_label = "Month"
        plot.yaxis.axis_label = unit
        plot.legend.location = "bottom_center"
        plot.legend.click_policy = "mute"
        plot.legend.orientation = "horizontal"
        if sum(locationDF[groupname][varname]) > 0:
            legendItems.append((varname, [currentLine]))
    legend = Legend(items=legendItems)
    plot.add_layout(legend, 'right')
    show(plot)

def plotByVariable(variable):
#     cols = baseCols+[variable]
    locationDF = df[df["Type"] == variable]
    locationDF.fillna(0)
    print(locationDF)
    locationDF = locationDF.pivot(index='Month', columns=variable)
    recyclingTypes = list(locationDF.columns)
    source = ColumnDataSource(locationDF)
    colNames = source.column_names

    

In [3]:
colors = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#ffff33", "#a65628", "#f781bf", "#999999"]
    
catalog = intake.open_catalog('../catalog.yml')


unitDict = []
unit = "lbs"

for table in catalog:
    if table == "landfill" or table == "recycling":
        continue
    df = catalog[table].read()
    print(df.columns, df["Type"].unique())
    for location in df.columns[1:-1]:
        plotAtLocation(df, location)

Index(['Type', 'Mulch', 'Month'], dtype='object') ['Leaves' 'Brush']
Mulch not in dict


/anaconda3/envs/hackathon/lib/python3.6/site-packages/bokeh/models/plots.py:725: UserWarning: 
You are attemptings to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/anaconda3/envs/hackathon/lib/python3.6/site-packages/bokeh/models/plots.py:725: UserWarning: 
You are attemptings to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/anaconda3/envs/hackathon/lib/python3.6/site-packages/bokeh/models/plots.py:725: UserWarning: 
You are attemptings to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legen

Index(['Type', 'Totals', 'Percents', 'Month'], dtype='object') ['Commodity Recycled' 'SWMF Recycled' 'Mulch Recycled'
 'C&D (class 3) Diverted' 'Garbage (class 1)']
Totals not in dict
Percents not in dict
Index(['Type', 'Waste', 'C&D, Diverted', 'Month'], dtype='object') ['Transfer Station ' 'Household Trash' 'Misc Trash' 'Construction' 'Codes']
Waste not in dict
C&D, Diverted not in dict
Index(['Type', 'Magnolia & Alice', '225 Moody', '4440 Western Av.',
       '341 Parkvillage', '227 Willow Av.', 'Curbside City-Wide', 'Downtown',
       'KPD', 'Recycling Trailer', 'Month'],
      dtype='object') ['Aluminum' 'Plastics ("Commingled")' 'Glass' 'Mixed Paper' 'Cardboard'
 'Steel/Tin' 'Newspaper' 'All Mixed (no glass)']
Index(['Type', 'Non-Commodity Recycling', 'Diverted', 'Month'], dtype='object') ['TS Scrap Metal*' 'TS Tires' 'TS Carpet' 'HHW E-Waste'
 'HHW Mixed Recycling' 'HHW Diverted']
Non-Commodity Recycling not in dict
Diverted not in dict
